# Processing mixed beverage data
This Jupyter Notebook uses curl to down download [Mixed Beverage Gross Receipts](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php) files from the Texas Comptroller's [data center](https://comptroller.texas.gov/transparency/open-data/search-datasets/), and then a python library called [agate](http://agate.readthedocs.io/) to clean and process that data for [stories similar to this one](http://www.mystatesman.com/business/austin-alcohol-sales-percent-february/Oo2txZUkuDlqBl0rU9O1lJ/) on monthly alcohol sales.

This is a stripped down version (compared to the original fork) that skips explanation of steps beyond commenting.

## Get to the goods

To skip most of the setup code and get to what you really want, search for:

- Top sales statewide
- Austin sales
- Central Texas cities

### File download

- Go to the [Texas Comptroller data center](https://comptroller.texas.gov/transparency/open-data/search-datasets/) and copy the url for the CSV for this month and enter it below.
- You also need to set to set the [processing variables](Processing-variables) for this month.

In [1]:
## %%bash
## downloads the mixedbev file into mixbev-files folder
## You have to set this URL based on location in data center

## these are commented out now. I've been pulling them manually
## and processing from those file
# cd ../test/
# curl -O https://comptroller.texas.gov/auto-data/odc/MIXEDBEV_03_2017.CSV

In [2]:
# imports the libraries we will use
import agate
from decimal import Decimal
import re

# this surpresses the timezone warning
# Might comment out during development so other warnings
# are not surpressed
import warnings
warnings.filterwarnings('ignore')

### Processing variables
Then we set some values based on those.

- The **`file`** is the name of the file we want to process
- The **`tax_rate`** is the value we need for this file to get the Gross Recipts (vs the Tax Reported, which is just the tax amount the establishment paid). The comptroller [has information on the tax](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php), but this [old record layout](https://github.com/utdata/cli-tools/blob/master/data/mixbevtax/OLD-MIXEDBEVTAX-LAYOUT.txt) best describes the math.
- The **`month_studied`** is the YYYY/MM designation for the month before the file release. The file released in February has mostly records from January, but can also have any other month, so we set here the specific month we want. Note there is a check later on that counts the number of files by month, which is worth checking.

Here are the files we have downloaded:

In [3]:
ls ../mixbev-files/2017/

MIXEDBEV_01_2017.CSV  MIXEDBEV_04_2015.CSV  MIXEDBEV_06_2017.CSV
MIXEDBEV_02_2017.CSV  MIXEDBEV_04_2017.CSV
MIXEDBEV_03_2017.CSV  MIXEDBEV_05_2017.CSV


In [4]:
# this is our source file, which may have been downloaded above
# Swap out the file name here and date below as needed
file = '../mixbev-files/2017/MIXEDBEV_06_2017.CSV'
file_name = ''
online_name = ''

# setting the month_studied var.
# This should be checked in the table below that counts records by month
month_studied = '2017/05'

# Sets the tax rate to convert Report Tax to Gross Receipts
# It's 6.7 since January 1, 2014
tax_rate = Decimal('6.7')

### import and processing

In [5]:
# sets the column names of the original data set.
column_names = [
    'TABC Permit Number',
    'Trade Name',
    'Location Address',
    'Location City',
    'Location State',
    'Location Zip Code',
    'Location County Code',
    'Blank',
    'Report Period',
    'Report Tax'
]
# Helps us import some text fields that may be considered numbers in error.
specified_types = {
    'Location Zip Code': agate.Text(),
    'Location County Code': agate.Text()
}

# this imports the file specified above, along with the proper types
mixbev_raw = agate.Table.from_csv(file, column_names, encoding='iso-8859-1', column_types=specified_types)

# mixbev_trim creates a new interim table with results of compute function
# that takes the four columns that need trimming and strips them of white space,
# adding them to the end of the table with new names.
# The last computation does the math to create the Gross Receipts based on the tax_rate set above

mixbev_trim = mixbev_raw.compute([
    ('Permit', agate.Formula(agate.Text(), lambda r: r['TABC Permit Number'].strip())),
    ('Name', agate.Formula(agate.Text(), lambda r: r['Trade Name'].strip())),
    ('Address', agate.Formula(agate.Text(), lambda r: r['Location Address'].strip())),
    ('City', agate.Formula(agate.Text(), lambda r: r['Location City'].strip())),
    ('Receipts_compute', agate.Formula(agate.Number(), lambda r: (r['Report Tax'] / tax_rate) * 100))
])

# the Receipts_compute computation above returns as a decimal number,
# so this function rounds those numbers.
# I might refactor this late so I can use it elsewhere.
def round_receipt(row):
    return row['Receipts_compute'].quantize(Decimal('0.01'))

# This compute method uses round_recipt function above,
# putting the results into a new table.
mixbev_round = mixbev_trim.compute([
    ('Receipts', agate.Formula(agate.Number(), round_receipt))
])

# creates new table, selecting just the columns we need
# then renames some of them for ease later.
mixbev_cleaned = mixbev_round.select([
    'Permit',
    'Name',
    'Address',
    'City',
    'Location State',
    'Location Zip Code',
    'Location County Code',
    'Report Period',
    'Report Tax',
    'Receipts'
]).rename(column_names = {
    'Location State': 'State',
    'Location Zip Code': 'Zip',
    'Location County Code': 'CountyCode',
    'Report Period': 'Period',
    'Report Tax': 'Tax'
})

# Concatenates the name and address
mixbev_cleaned_est = mixbev_cleaned.compute([
    ('Establishment', agate.Formula(agate.Text(), lambda row: '%(Name)s %(Address)s' % row))
])

# importing countes.csv, ensuring that the 'code' column is text
counties = agate.Table.from_csv('../resource-files/counties.csv', column_types={'code': agate.Text()})

# joines the counties table to the mixed bev cleaned data with establishments
mixbev_joined = mixbev_cleaned_est.join(counties, 'CountyCode', 'code')

# get just the columns we need and rename county
# THIS is the finished, cleaned mixbev table
mixbev = mixbev_joined.select([
    'Permit',
    'Name',
    'Address',
    'Establishment',
    'City',
    'State',
    'Zip',
    'county',
    'Period',
    'Tax',
    'Receipts'
]).rename(column_names = {
    'county': 'County'
})


### Looking at dates of the records

This basically confirms that the file has multiple dates, and that we are looking at the right month of data. Typically a data set will have mostly reports from the previous month, but there are always also submissions from other months. We want to filter out those other months, which we do based on the `month_studied` variable set near the top of the file, which should match the period at the top of the table below.


In [6]:
# Pivot the mixbev table by Period. Default it give a Count of the records
# We then order the table by Count in descending order
by_period = mixbev.pivot('Period').order_by('Count', reverse=True)

# prints the table of period and number of records
by_period.limit(5).print_table(max_rows=None)

| Period  |  Count |
| ------- | ------ |
| 2017/05 | 14,469 |
| 2017/04 |  1,375 |
| 2017/03 |    173 |
| 2017/06 |     53 |
| 2017/02 |     48 |


In [7]:
## filters the records to our month_studied
mixbev_month = mixbev.where(lambda row: row['Period'] == month_studied)

# function to group sales by a specific location
# City or County passed in should be ALL CAPS
# Location_type can be 'City' or 'County'

def location_sum(location_type, location):
    # Filters the data to the specified city
    location_filtered = mixbev_month.where(lambda row: row[location_type].upper() == location)

    # groups the data based on Establishment and location
    location_grouped = location_filtered.group_by('Establishment').group_by(location_type)
    # computes the sales based on the grouping
    location_summary = location_grouped.aggregate([
        ('Tax_sum', agate.Sum('Tax')),
        ('Receipts_sum', agate.Sum('Receipts'))
    ])
    
    # sorts the results by most sold
    location_summary_sorted = location_summary.order_by('Receipts_sum', reverse=True)
    # prints the top 10 results
    
    return(location_summary_sorted)


## Top sales statewide

Because we want to group our results by more than one field and perform more than one aggregation, we'll do this a little differently. We'll use group_by to create a grouped table, then perform aggregations on that new table to computer the Tax and Receipts columns.

In [31]:
# groups the data based on Establishment and City
mixbev_grouped = mixbev_month.group_by('Establishment').group_by('County').group_by('City')

# computes the sales based on the grouping
state_summary = mixbev_grouped.aggregate([
    ('Tax_sum', agate.Sum('Tax')),
    ('Sales_sum', agate.Sum('Receipts'))
])

# sorts the results by most sold. We could probalby chain it above if we wanted to.
state_summary_sorted = state_summary.order_by('Sales_sum', reverse=True)

# summing sales statewide for month
print('\nTotal sales across the state for the given month: {}\n'.format(
    mixbev_month.aggregate(agate.Sum('Receipts'))
))

print('Top sales by establishment statewide\n')

# prints the top 10 results
state_summary_sorted.limit(10).print_table(max_column_width=35)


Total sales across the state for the given month: 553995994.97

Top sales by establishment statewide

| Establishment                       | County     | City            |    Tax_sum |    Sales_sum |
| ----------------------------------- | ---------- | --------------- | ---------- | ------------ |
| GAYLORD TEXAN 1501 GAYLORD TRL      | Tarrant    | GRAPEVINE       | 115,258.82 | 1,720,280.90 |
| SPECTRUM CATERING 27328 SPECTRUM... | Montgomery | OAK RIDGE NORTH |  87,064.82 | 1,299,474.93 |
| ARAMARK SPORTS & ENTERTAINMENT 5... | Harris     | HOUSTON         |  81,612.49 | 1,218,096.87 |
| FOUR SEASONS HOTEL AND RESORT 41... | Dallas     | IRVING          |  76,418.92 | 1,140,580.90 |
| SALC, INC. 2201 N STEMMONS FWY FL 1 | Dallas     | DALLAS          |  70,827.84 | 1,057,131.94 |
| LEVY RESTAURANTS AT TOYOTA CEN 1... | Harris     | HOUSTON         |  70,729.89 | 1,055,670.00 |
| WLS BEVERAGE CO 110 E 2ND ST        | Travis     | AUSTIN          |  65,968.87 |   984,610.00 |
| MARR

## Austin sales and sums

With this, we refernce the location_sum function above, and pass the type of location (City) and the name of the city (AUSTIN). At the same time, we limit the result of that function to the first 10 records, and then print the results. We are basically stringing together a bunch of stuff at once.

In [29]:
# uses the city_sum function to filter
austin = location_sum('City', 'AUSTIN')

print('\nTotal sales across the state for the given month: {}\n'.format(
    austin.aggregate(agate.Sum('Receipts_sum'))
))

# print the resulting table
austin.limit(10).print_table(max_column_width=50)


Total sales across the state for the given month: 63324707.07

| Establishment                                      | City   |   Tax_sum | Receipts_sum |
| -------------------------------------------------- | ------ | --------- | ------------ |
| WLS BEVERAGE CO 110 E 2ND ST                       | AUSTIN | 65,968.87 |   984,610.00 |
| 400 BAR/CUCARACHA/CHUPACABRA/J 400 E 6TH ST        | AUSTIN | 32,572.05 |   486,150.00 |
| ROSE ROOM/ 77 DEGREES 11500 ROCK ROSE AVE          | AUSTIN | 32,456.34 |   484,422.99 |
| TOP GOLF 2700 ESPERANZA XING                       | AUSTIN | 29,910.14 |   446,420.00 |
| SAN JACINTO BEVERAGE COMPANY L 98 SAN JACINTO BLVD | AUSTIN | 29,107.61 |   434,441.94 |
| OASIS CANTINA DEL LAGO 6550 COMANCHE TRL           | AUSTIN | 28,316.88 |   422,640.00 |
| THE DOGWOOD DOMAIN 11420 ROCK ROSE AVE STE 700     | AUSTIN | 27,231.41 |   406,438.96 |
| RYAN SANDERS SPORTS SERVICES, 9201 CIRCUIT OF T... | AUSTIN | 27,018.82 |   403,265.97 |
| W HOTEL AUSTIN 200 LAVAC

## More Central Texas cities

In [10]:
location_sum('City', 'BASTROP').limit(3).print_table(max_column_width=40)

| Establishment                            | City    |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------- | -------- | ------------ |
| OLD TOWN RESTURANT AND BAR/PIN 931 MA... | BASTROP | 3,952.59 |    58,993.88 |
| CHILI'S GRILL & BAR 734 HIGHWAY 71 W     | BASTROP | 2,836.71 |    42,338.96 |
| NEIGHBOR'S 601 CHESTNUT ST UNIT C        | BASTROP | 2,456.22 |    36,660.00 |


In [11]:
location_sum('City', 'BEE CAVE').limit(3).print_table(max_column_width=40)

| Establishment                            | City     |  Tax_sum | Receipts_sum |
| ---------------------------------------- | -------- | -------- | ------------ |
| WOODY TAVERN AND GRILL, INC. 12801 SH... | BEE CAVE | 6,658.52 |    99,380.90 |
| MAUDIE'S HILL COUNTRY, LLC 12506 SHOP... | BEE CAVE | 6,004.40 |    89,617.91 |
| CAFE BLUE 12800 HILL COUNTRY BLVD STE    | BEE CAVE | 5,921.59 |    88,381.94 |


In [12]:
location_sum('City', 'BUDA').limit(3).print_table(max_column_width=40)

| Establishment                            | City |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---- | -------- | ------------ |
| BUCKS BACKYARD 1750 S FM 1626            | BUDA | 6,908.10 |   103,105.97 |
| WILLIE'S JOINT 824 MAIN ST               | BUDA | 3,858.93 |    57,595.97 |
| NATE'S AT BUDA MILL AND GRAIN, 306 S ... | BUDA | 3,250.37 |    48,512.99 |


In [13]:
location_sum('City', 'CEDAR PARK').limit(3).print_table(max_column_width=40)

| Establishment                            | City       |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | -------- | ------------ |
| CHUY'S 4911 183A TOLL RD                 | CEDAR PARK | 9,185.09 |   137,090.90 |
| LUPE TORTILLA MEXICAN RESTAURA 4501 1... | CEDAR PARK | 7,778.56 |   116,097.91 |
| BLUE CORN HARVEST BAR & GRILL 700 E W... | CEDAR PARK | 5,459.76 |    81,488.96 |


In [14]:
location_sum('City', 'DRIPPING SPRINGS').limit(3).print_table(max_column_width=40)

| Establishment                            | City             |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------------- | -------- | ------------ |
| DEEP EDDY DISTILLING CO 2250 E HIGHWA... | DRIPPING SPRINGS | 4,381.13 |    65,390.00 |
| FLORES MEXICAN RESTAURANT 2440 E HIGH... | DRIPPING SPRINGS | 4,059.73 |    60,592.99 |
| TRUDY'S FOUR STAR 13059 FOUR STAR BLVD   | DRIPPING SPRINGS | 3,995.94 |    59,640.90 |


In [15]:
location_sum('City', 'GEORGETOWN').limit(3).print_table(max_column_width=40)

| Establishment                      | City       |  Tax_sum | Receipts_sum |
| ---------------------------------- | ---------- | -------- | ------------ |
| EL MONUMENTO 205 W 2ND ST          | GEORGETOWN | 6,923.24 |   103,331.94 |
| HARDTAILS 1515 N IH 35             | GEORGETOWN | 4,968.58 |    74,157.91 |
| DOS SALSAS CAFE INC 1104 S MAIN ST | GEORGETOWN | 4,936.76 |    73,682.99 |


In [16]:
location_sum('City', 'KYLE').limit(3).print_table(max_column_width=40)

| Establishment                            | City |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---- | -------- | ------------ |
| CASA GARCIA'S MEXICAN RESTAURA 5401 F... | KYLE | 5,527.43 |    82,498.96 |
| EVO ENTERTAINMENT CENTER 3200 KYLE XING  | KYLE | 3,628.45 |    54,155.97 |
| CENTERFIELD SPORTS BAR & GRILL 200 W ... | KYLE | 2,681.00 |    40,014.93 |


In [17]:
location_sum('City', 'LAGO VISTA').limit(3).print_table(max_column_width=40)

| Establishment                            | City       |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | -------- | ------------ |
| COPPERHEAD GRILL 6115 LOHMANS FORD RD    | LAGO VISTA | 1,141.88 |    17,042.99 |
| MARIA'S BAR & GRILL MEXICAN RE 20602 ... | LAGO VISTA |   409.90 |     6,117.91 |


In [18]:
location_sum('City', 'LAKEWAY').limit(3).print_table(max_column_width=40)

| Establishment                            | City    |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------- | -------- | ------------ |
| LAKEWAY RESORT AND SPA 101 LAKEWAY DR    | LAKEWAY | 9,109.11 |   135,956.87 |
| THE GROVE WINE BAR AND KITCHEN 3001 R... | LAKEWAY | 7,711.16 |   115,091.94 |
| FLORES MEXICAN RESTAURANT 2127 LOHMAN... | LAKEWAY | 4,267.29 |    63,690.90 |


In [19]:
location_sum('City', 'LEANDER').limit(3).print_table(max_column_width=40)

| Establishment                            | City    |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------- | -------- | ------------ |
| BROOKLYN HEIGHTS PIZZERIA 3550 LAKELI... | LEANDER | 3,807.87 |    56,833.88 |
| JARDIN DEL REY 703 S HIGHWAY 183         | LEANDER | 2,982.90 |    44,520.90 |
| TAPATIA JALISCO #3 LLC 651 N US 183      | LEANDER |   930.16 |    13,882.99 |


In [20]:
location_sum('City', 'LIBERTY HILL').limit(3).print_table(max_column_width=40)

| Establishment                            | City         |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------------ | -------- | ------------ |
| JARDIN CORONA 15395 W STATE HIGHWAY 29   | LIBERTY HILL | 3,092.65 |    46,158.96 |
| MARGARITA'S RESTAURANT 10280 W STATE ... | LIBERTY HILL | 2,922.94 |    43,625.97 |
| FIRE OAK DISTILLERY 4600 COUNTY ROAD 207 | LIBERTY HILL |     0.00 |         0.00 |


In [21]:
location_sum('City', 'PFLUGERVILLE').limit(3).print_table(max_column_width=40)

| Establishment                            | City         |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------------ | -------- | ------------ |
| MAVERICKS 1700 GRAND AVENUE PKWY STE 2   | PFLUGERVILLE | 7,891.93 |   117,790.00 |
| HANOVER'S DRAUGHT HAUS 108 E MAIN ST     | PFLUGERVILLE | 4,628.15 |    69,076.87 |
| RIO GRANDE TEX MEX 15821 CENTRAL COMM... | PFLUGERVILLE | 4,066.83 |    60,698.96 |


In [22]:
location_sum('City', 'ROUND ROCK').limit(5).print_table(max_column_width=40)

| Establishment                            | City       |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | -------- | ------------ |
| THIRD BASE ROUND ROCK, LLC 3107 S INT... | ROUND ROCK | 9,567.53 |   142,798.96 |
| JACK ALLEN'S KITCHEN 2500 HOPPE TRL      | ROUND ROCK | 9,474.67 |   141,412.99 |
| CHUY'S ROUND ROCK 2320 N INTERSTATE 35   | ROUND ROCK | 8,780.35 |   131,050.00 |
| TWIN PEAKS RESTAURANT 100 LOUIS HENNA... | ROUND ROCK | 8,436.84 |   125,922.99 |
| RICK'S CABARET 3105 S INTERSTATE 35      | ROUND ROCK | 8,322.33 |   124,213.88 |


In [23]:
location_sum('City', 'SAN MARCOS').limit(5).print_table(max_column_width=40)

| Establishment                            | City       |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ---------- | -------- | ------------ |
| ZELICKS 336 W HOPKINS ST                 | SAN MARCOS | 9,739.38 |   145,363.88 |
| THE MARC 120 E SAN ANTONIO ST            | SAN MARCOS | 8,746.58 |   130,545.97 |
| CHIMY'S SAN MARCOS 217 E HOPKINS ST      | SAN MARCOS | 8,147.66 |   121,606.87 |
| PLUCKERS WING BAR 105 N INTERSTATE 35    | SAN MARCOS | 7,028.90 |   104,908.96 |
| BLACK RABBIT SALOON / SHADE RO 127 E ... | SAN MARCOS | 7,017.04 |   104,731.94 |


In [24]:
location_sum('City', 'SPICEWOOD').limit(3).print_table(max_column_width=40)

| Establishment                            | City      |  Tax_sum | Receipts_sum |
| ---------------------------------------- | --------- | -------- | ------------ |
| ANGEL'S ICEHOUSE 21815 W HWY 71          | SPICEWOOD | 4,192.12 |    62,568.96 |
| POODIES HILLTOP ROADHOUSE 22308 STATE... | SPICEWOOD | 3,030.34 |    45,228.96 |
| APIS RESTAURANT 23526 STATE HIGHWAY 71 W | SPICEWOOD | 1,915.93 |    28,595.97 |


In [25]:
location_sum('City', 'SUNSET VALLEY').limit(3).print_table(max_column_width=40)

| Establishment                            | City          |  Tax_sum | Receipts_sum |
| ---------------------------------------- | ------------- | -------- | ------------ |
| DOC'S BACKYARD 5207 BRODIE LN # 100      | SUNSET VALLEY | 5,391.95 |    80,476.87 |
| BJ'S RESTAURANT AND BREWHOUSE 5207 BR... | SUNSET VALLEY | 4,212.29 |    62,870.00 |
| CANE ROSSO 4715 S LAMAR BLVD STE 103     | SUNSET VALLEY | 2,038.47 |    30,424.93 |


In [26]:
location_sum('City', 'WEST LAKE HILLS').limit(3).print_table(max_column_width=40)

| Establishment                            | City            |  Tax_sum | Receipts_sum |
| ---------------------------------------- | --------------- | -------- | ------------ |
| LUPE TORTILLA MEXICAN RESTAURA 701 S ... | WEST LAKE HILLS | 6,786.16 |   101,285.97 |
| CHIPOTLE CHIPOTLE MEXICAN GRIL 3300 B... | WEST LAKE HILLS |    36.78 |       548.96 |


## Sales by county example

In this case, we pass in the location type of 'County' and then a county name in caps to get the most sales in a particular county.

In [27]:
location_sum('County', 'CALDWELL').limit(3).print_table(max_column_width=40)

| Establishment                            | County   |  Tax_sum | Receipts_sum |
| ---------------------------------------- | -------- | -------- | ------------ |
| GUADALAJARA MEXICAN RESTAURANT 1710 S... | Caldwell | 1,274.47 |    19,021.94 |
| THE PEARL 110 N MAIN ST                  | Caldwell |   695.72 |    10,383.88 |
| MR TACO 1132 E PIERCE ST                 | Caldwell |   528.96 |     7,894.93 |


## Sales by ZIP Code
Just making sure that 78701 is at the top of this list, which it has been every month for a decade.

In [28]:
# top zip code gross receipts
zip_receipts = mixbev_month.pivot('Zip', aggregation=agate.Sum('Receipts')).order_by('Sum', reverse=True)
zip_receipts.limit(5).print_table()

| Zip   |           Sum |
| ----- | ------------- |
| 78701 | 23,441,262.88 |
| 75201 | 11,874,852.15 |
| 77002 |  9,964,745.01 |
| 78205 |  9,743,927.72 |
| 78704 |  7,650,495.57 |
